In [1]:
import pandas as pd
import fasttext
import fasttext_models

execfile('fasttext_models.py')

NameError: name 'execfile' is not defined

In [28]:
# load the models for evaluation
m0 = fasttext.load_model('fasttext_skipgram_cleaned.bin')
m1 = fasttext.load_model('fasttext_skipgram_uncleaned.bin')
print("Fasttext classifiers store words by most to least frequent.")
print("\nHere are the top 50 words for the model trained on cleaned data:")
m0_words = m0.get_words()
print(m0_words[:50])
print("\nHere are the top 50 words for the model trained on uncleaned data:")
m1_words = m1.get_words()
print(m1_words[:50])

Fasttext classifiers store words by most to least frequent.

Here are the top 50 words for the model trained on cleaned data:
['>', '<', 'br', '</s>', 'like', 'good', 'taste', 'flavor', 'one', 'get', 'love', 'product', 'make', 'use', 'coffee', 'great', 'try', 'well', 'food', 'buy', 'tea', 'find', 'would', 'eat', 'dog', 'go', 'really', 'time', 'much', 'amazon', 'order', 'also', 'price', 'bag', 'cup', 'give', 'little', 'even', 'drink', 'say', 'think', 'store', 'day', 'cat', 'add', 'box', 'chocolate', 'treat', 'come', 'first']

Here are the top 50 words for the model trained on uncleaned data:
['the', 'I', 'and', 'a', 'to', 'of', 'is', 'it', '</s>', 'for', 'in', 'this', 'that', 'my', 'with', 'have', 'but', 'are', 'was', 'not', 'you', '/><br', 'on', 'as', 'like', 'they', 'so', 'be', 'The', 'or', 'at', 'these', 'just', 'them', 'very', 'from', 'one', 'good', 'It', '"I', 'has', 'can', 'taste', 'will', 'would', 'had', 'all', 'more', 'than', 'when']


In [20]:
print("We can also measure the accuracy of these classfiers by examining their precision and recall when running on a test set.\n")

n0,p0,r0 = m0.test('reviews_cleaned.test')
n1,p1,r1 = m1.test('reviews_uncleaned.test')
d = {'Model':['Cleaned', 'Uncleaned'], 'N':[n0,n1], 'Precision':[p0,p1],'Recall':[r0,r1]}
df = pd.DataFrame(data=d)
print(df)

print("\nRemarkably, though we know the two models are different, they have identical precision and recall.")

We can also measure the accuracy of these classfiers by examining their precision and recall when running on a test set.

       Model       N  Precision    Recall
0    Cleaned  113691   0.737754  0.737754
1  Uncleaned  113691   0.778417  0.778417

Remarkably, though we know the two models are different, they have identical precision and recall.


In [31]:
print("Now let's take a look at some of the vector embeddings.")
print("Let's look at the vector for \"chocolate\" in the cleaned-data model:\n")
print(m0.get_word_vector('chocolate'))
print("\nWe can also look at the most similar words, or the vector's 'neighbors.' These are determined by the cosine similarity of vectors.")
m0.get_nearest_neighbors('chocolate')

Now let's take a look at some of the vector embeddings.
Let's look at the vector for "chocolate" in the cleaned-data model:

[-0.02479652  0.00130697  0.0024827  -0.01109868  0.02672165 -0.01601592
  0.02024821  0.02821331 -0.03821959 -0.02994298 -0.00793434 -0.05219461
 -0.05966956 -0.02902482  0.0145936  -0.00822408  0.0551844   0.04180275
  0.00351004  0.0683969  -0.01808743 -0.0510674  -0.02598811  0.02973687
  0.09353215 -0.01456618 -0.04731614 -0.02148609 -0.0252507  -0.00568354
  0.02244259  0.00751527 -0.00421518  0.0400953   0.03187301  0.05662943
 -0.0196273   0.05481498 -0.05990427  0.0099687   0.04064913 -0.00840833
 -0.04372537 -0.02308569 -0.00385587 -0.01340246 -0.04709338 -0.03202996
 -0.02764771 -0.05866976]

We can also look at the most similar words, or the vector's 'neighbors.' These are determined by the cosine similarity of vectors.


[(0.8768860697746277, 'jerky'),
 (0.8577314615249634, 'chai'),
 (0.8398478627204895, 'invisible'),
 (0.822700560092926, '97'),
 (0.8224048614501953, 'kavli'),
 (0.8212175369262695, 'soaking'),
 (0.8192944526672363, 'yor'),
 (0.8187270760536194, 'nation'),
 (0.8184553384780884, 'possibilitie'),
 (0.817771315574646, 'pastey')]

In [ ]:
print("WOWEEE!")

# maybe here we can try out some vector addition/subtraction to see what kind of meaning the model captures. 